In [1]:
import numpy as np
import os
from json_tricks import dump, load

from pydub import AudioSegment, effects

import librosa
import tensorflow as tf
import keras
import sklearn

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Emotion kind validation function for TESS database, due to emotions written within the file names.
def find_emotion_T(name): 
        if('neutral' in name): return "01"
        elif('happy' in name): return "03"
        elif('sad' in name): return "04"
        elif('angry' in name): return "05"
        elif('fear' in name): return "06"
        elif('disgust' in name): return "07"
        elif('ps' in name): return "08"
        else: return "-1"
        
        
# 'emotions' list fix for classification purposes:
#     Classification values start from 0, Thus an 'n = n-1' operation has been executed for both RAVDESS and TESS databases:
def emotionfix(e_num):
    if e_num == "01":   return 0 # neutral
    elif e_num == "02": return 1 # calm
    elif e_num == "03": return 2 # happy
    elif e_num == "04": return 3 # sad
    elif e_num == "05": return 4 # angry
    elif e_num == "06": return 5 # fear
    elif e_num == "07": return 6 # disgust
    else:               return 7 # suprised

In [3]:
sample_lengths = []
folder_path = './dataset'

for subdir, dirs, files in os.walk(folder_path):
  for file in files: 
    x, sr = librosa.load(path = os.path.join(subdir,file), sr = None)
    xt, index = librosa.effects.trim(x, top_db=30)
     
    sample_lengths.append(len(xt))

print('Maximum sample length:', np.max(sample_lengths))        

Maximum sample length: 204288


In [12]:
import time
import noisereduce as nr
tic = time.perf_counter()

# Initialize data lists
rms = []
zcr = []
mfcc = []
emotions = []
fileName = []

# Initialize variables
total_length = 204288 # desired frame length for all of the audio samples.
frame_length = 2048
hop_length = 512

folder_path = './dataset' 

for subdir, dirs, files in os.walk(folder_path):
  for file in files: 

    # Fetch the sample rate.
      _, sr = librosa.load(path = os.path.join(subdir,file), sr = None) # sr (the sample rate) is used for librosa's MFCCs. '_' is irrelevant.
        
# Load the audio file.
      rawsound = AudioSegment.from_file(os.path.join(subdir,file)) 
    # Normalize the audio to +5.0 dBFS.
      normalizedsound = effects.normalize(rawsound, headroom = 5.0) 
    # Transform the normalized audio to np.array of samples.
      normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32')
    # Trim silence from the beginning and the end.
      xt, index = librosa.effects.trim(normal_x, top_db=30)
      #print(file,"\t", len(xt), "\t", rawsound.dBFS, "\t", normalizedsound.dBFS) #--QA purposes if needed-- 
    # Pad for duration equalization.
      padded_x = np.pad(xt, (0, total_length-len(xt)), 'constant')
    # Noise reduction.
      final_x = nr.reduce_noise(y=padded_x,
                          sr=sr)
       
   # Features extraction 
      f1 = librosa.feature.rms(y=final_x, frame_length=frame_length, hop_length=hop_length) # Energy - Root Mean Square   
      f2 = librosa.feature.zero_crossing_rate(y=final_x , frame_length=frame_length, hop_length=hop_length, center=True) # ZCR      
      f3 = librosa.feature.mfcc(y=final_x, sr=sr, n_mfcc=13, hop_length = hop_length) # MFCC
      
   # Emotion extraction from the different databases
      if (find_emotion_T(file) != "-1"): #TESS database validation
            name = find_emotion_T(file)
      else:                              #RAVDESS database validation
            name = file[6:8]                      

   # Filling the data lists  
      rms.append(f1)
      zcr.append(f2)
      mfcc.append(f3)
      fileName.append(file)
      #print(file, emotionfix(name))
      emotions.append(emotionfix(name)) 

toc = time.perf_counter()
print(f"Running time: {(toc - tic)/60:0.4f} minutes")

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:

f_rms = np.asarray(rms).astype('float32')
f_rms = np.swapaxes(f_rms,1,2)
f_zcr = np.asarray(zcr).astype('float32')
f_zcr = np.swapaxes(f_zcr,1,2)
f_mfccs = np.asarray(mfcc).astype('float32')
f_mfccs = np.swapaxes(f_mfccs,1,2)

print('ZCR shape:',f_zcr.shape)
print('RMS shape:',f_rms.shape)
print('MFCCs shape:',f_mfccs.shape)

In [ ]:
# Concatenating all features to 'X' variable.
X = np.concatenate((f_zcr, f_rms, f_mfccs), axis=2)

# Preparing 'Y' as a 2D shaped variable.
Y = np.asarray(emotions).astype('int8')
Y = np.expand_dims(Y, axis=1)

In [ ]:
# Split to train, validation, and test sets.
from sklearn.model_selection import train_test_split
x_train, x_tosplit, y_train, y_tosplit = train_test_split(X, Y, test_size = 0.224, random_state = 1,shuffle=True)
x_val, x_test, y_val, y_test = train_test_split(x_tosplit, y_tosplit, test_size = 0.304, random_state = 1,shuffle=True)

#'One-hot' vectors for Y: emotion classification
y_train_class = tf.keras.utils.to_categorical(y_train, 8, dtype = 'int8')
y_val_class = tf.keras.utils.to_categorical(y_val, 8, dtype = 'int8')

In [ ]:
# x_train, x_val, and x_test shape check.
print(np.shape(x_train))
print(np.shape(x_val))
print(np.shape(x_test))
print(np.shape(Y))

In [ ]:
import os
from json_tricks import load

import numpy as np

import librosa
from pydub import AudioSegment, effects
import noisereduce as nr

import tensorflow as tf
import keras
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

In [ ]:
saved_model_path  = f'./model.json'
saved_weights_path = f'./model_weights.h5'
#Reading the model from JSON file
with open(saved_model_path, 'r') as json_file:
    json_savedModel = json_file.read()
    
# Loading the model architecture, weights
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights(saved_weights_path)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)   
# Compiling the model with similar parameters as the original model.
model.compile(loss='categorical_crossentropy', 
                optimizer=opt, 
                metrics=['categorical_accuracy'])

print(model.summary())

In [ ]:
# Validation score
print("Evaluate data size : " , np.shape(x_val)[0])
loss,acc = model.evaluate(x_val, y_val_class, verbose=2)
print("Accuracy", str(acc * 100) + "%")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
# Validation Confusion matrix
import pandas as pd
y_val_class = np.argmax(y_val_class, axis=1)
predictions = model.predict(x_val)
y_pred_class = np.argmax(predictions, axis=1)

cm=confusion_matrix(y_val_class, y_pred_class)

index = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']  
columns = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']  
 
cm_df = pd.DataFrame(cm,index,columns)                      
plt.figure(figsize=(12,8))
ax = plt.axes()

sns.heatmap(cm_df, ax = ax, cmap = 'PuBu', fmt="d", annot=True)
ax.set_ylabel('True emotion')
ax.set_xlabel('Predicted emotion')

In [ ]:
values = cm.diagonal()
row_sum = np.sum(cm,axis=1)
acc = values / row_sum

print('Validation set predicted emotions accuracy:')
for e in range(0, len(values)):
    print(index[e],':', f"{(acc[e]):0.4f}")

In [ ]:
print(x_val[0][3],y_val_class[3])

In [ ]:
predictions = model.predict(x_val, use_multiprocessing=True)

In [ ]:
predictions.shape

In [ ]:
x_val.shape

In [ ]:
np.argmax(predictions[3])

In [ ]:
emotions = {
    0 : 'neutral',
    1 : 'calm',
    2 : 'happy',
    3 : 'sad', #SAD
    4 : 'angry',
    5 : 'fearful',
    6 : 'disgust',
    7 : 'suprised'   
}
emo_list = list(emotions.values())

In [ ]:
def preprocess(file_path, subdir):
    rms_t = []
    zcr_t = []
    mfcc_t = []
    
    _, sr = librosa.load(path = os.path.join(subdir,file_path), sr = None) # sr (the sample rate) is used for librosa's MFCCs. '_' is irrelevant.
    # Load the audio file.
    rawsound = AudioSegment.from_file( os.path.join(subdir,file_path)) 
    # Normalize the audio to +5.0 dBFS.
    normalizedsound = effects.normalize(rawsound, headroom = 5.0) 
    # Transform the normalized audio to np.array of samples.
    normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32')
    # Trim silence from the beginning and the end.
    xt, index = librosa.effects.trim(normal_x, top_db=30)
      #print(file,"\t", len(xt), "\t", rawsound.dBFS, "\t", normalizedsound.dBFS) #--QA purposes if needed-- 
    # Pad for duration equalization.
    padded_x = np.pad(xt, (0, total_length-len(xt)), 'constant')
    # Noise reduction.
    final_x = nr.reduce_noise(y=padded_x,
                          sr=sr)
 
   # Features extraction 
    f1 = librosa.feature.rms(y=final_x, frame_length=frame_length, hop_length=hop_length) # Energy - Root Mean Square   
    f2 = librosa.feature.zero_crossing_rate(y=final_x , frame_length=frame_length, hop_length=hop_length, center=True) # ZCR      
    f3 = librosa.feature.mfcc(y=final_x, sr=sr, n_mfcc=13, hop_length = hop_length) # MFCC
      
   # Emotion extraction from the different databases
    if (find_emotion_T(file_path) != "-1"): #TESS database validation
        name = find_emotion_T(file_path)
    else:                              #RAVDESS database validation
        name = file_path[6:8]  

   # Filling the data lists  
    rms_t.append(f1)
    zcr_t.append(f2)
    mfcc_t.append(f3)
    
    f_rms_t = np.asarray(rms_t).astype('float32')
    f_rms_t = np.swapaxes(f_rms_t,1,2)
    f_zcr_t = np.asarray(zcr_t).astype('float32')
    f_zcr_t = np.swapaxes(f_zcr_t,1,2)
    f_mfccs_t = np.asarray(mfcc_t).astype('float32')
    f_mfccs_t = np.swapaxes(f_mfccs_t,1,2)

    #print(file, emotionfix(name))


    X = np.concatenate((f_zcr_t, f_rms_t, f_mfccs_t), axis=2)

    predictions = model.predict(X, use_multiprocessing=True)

    y_pred_class = np.argmax(predictions)
    print("file : " + str(file_path), ",Real emotion : " + str(emotionfix(name)) + " " + emotions.get(emotionfix(name),-1), ",predictions : " + str(y_pred_class)+ " " +emotions.get(y_pred_class,-1))
    return y_pred_class
    

In [ ]:
a = preprocess('03-01-01-01-01-01-02.wav','./dataset/Actor_02')

In [ ]:
a

In [ ]:
folder_path = './dataset' 

for subdir, dirs, files in os.walk(folder_path):
  for file in files: 
    b = preprocess(file,subdir)

In [ ]:
import pyaudio
import wave
from array import array
import struct
import time
from datetime import datetime
import shutil

In [ ]:
def voice_predictions():
    
    
    RATE = 24414
    CHUNK = 512
    RECORD_SECONDS = 7
    FORMAT = pyaudio.paInt32
    CHANNELS = 1
    WAVE_OUTPUT_FILE = "./output_testing.wav"
    WAVE_OUTPUT_FILENAME_TESTING = "./dataset/Actor_02/03-01-01-01-01-01-02.wav"
    
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    
    data = array('h', np.random.randint(size = 512, low = 0, high = 500))
    
    print("** Analysis started")
    total_predictions = [] # A list for all predictions in the session.
    tic = time.perf_counter()
    

    print("* Analysis...")
    frames = [] 
    data = np.nan # Reset 'data' variable.

    timesteps = int(RATE / CHUNK * RECORD_SECONDS) # => 339

    # Insert frames to 'output.wav'.
    for i in range(0, timesteps):
        data = array('l', stream.read(CHUNK)) 
        frames.append(data)

        wf = wave.open(WAVE_OUTPUT_FILE, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print("* done Analysis")

    rms_t = []
    zcr_t = []
    mfcc_t = []
    
    _, sr = librosa.load(path = WAVE_OUTPUT_FILENAME_TESTING, sr = None) # sr (the sample rate) is used for librosa's MFCCs. '_' is irrelevant.
    # Load the audio file.
    rawsound = AudioSegment.from_file(WAVE_OUTPUT_FILENAME_TESTING) 
    # Normalize the audio to +5.0 dBFS.
    normalizedsound = effects.normalize(rawsound, headroom = 0) 
    # Transform the normalized audio to np.array of samples.
    normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32')
    # Trim silence from the beginning and the end.
    xt, index = librosa.effects.trim(normal_x, top_db=30)
      #print(file,"\t", len(xt), "\t", rawsound.dBFS, "\t", normalizedsound.dBFS) #--QA purposes if needed-- 
    # Pad for duration equalization.
    padded_x = np.pad(xt, (0, 204288-len(xt)), 'constant')
    # Noise reduction.
    final_x = nr.reduce_noise(y=padded_x,
                          sr=sr)
 
    # Features extraction 
    f1 = librosa.feature.rms(y=final_x, frame_length=frame_length, hop_length=hop_length) # Energy - Root Mean Square   
    f2 = librosa.feature.zero_crossing_rate(y=final_x , frame_length=frame_length, hop_length=hop_length, center=True) # ZCR      
    f3 = librosa.feature.mfcc(y=final_x, sr=sr, n_mfcc=13, hop_length = hop_length) # MFCC
      
    # Filling the data lists  
    rms_t.append(f1)
    zcr_t.append(f2)
    mfcc_t.append(f3)
    
    f_rms_t = np.asarray(rms_t).astype('float32')
    f_rms_t = np.swapaxes(f_rms_t,1,2)
    f_zcr_t = np.asarray(zcr_t).astype('float32')
    f_zcr_t = np.swapaxes(f_zcr_t,1,2)
    f_mfccs_t = np.asarray(mfcc_t).astype('float32')
    f_mfccs_t = np.swapaxes(f_mfccs_t,1,2)

    #print(file, emotionfix(name))


    X = np.concatenate((f_zcr_t, f_rms_t, f_mfccs_t), axis=2)

    predictions = model.predict(X, use_multiprocessing=True)

    y_pred_class = np.argmax(predictions)
    print("predictions : " + str(y_pred_class)+ " " +emotions.get(y_pred_class))
    
    
    pred_list = list(predictions)
    pred_np = np.squeeze(np.array(pred_list).tolist(), axis=0) # Get rid of 'array' & 'dtype' statments.
    total_predictions.append(pred_np)
    print(pred_list)
    fig = plt.figure(figsize = (10, 2))
    plt.bar(emo_list, pred_np, color = 'darkturquoise')
    plt.ylabel("Probabilty (%)")
    plt.show()
    
    
    return predictions

In [ ]:
ABC = voice_predictions()
ABC